# Downloading FITS files for SOAR Telescope

## Startup Cells

In [1]:
# Import required libraries
import sys
import os
import pandas as pd
import numpy as np
import astropy.io.fits as pyfits
import requests
import json
from pprint import pprint as pp  # pretty print
from timeit import default_timer
import copy
import multiprocessing as mp
import mp_workers
from statistics import mean

In [99]:
# Set the API urls, check API version
url_root = "https://astroarchive.noao.edu/"
api_version = requests.get(f"{url_root}/api/version").text
print("Using API version {}".format(api_version))

Using API version 3.0


### Notes

While the valid instruments on the SOAR telescope include `osiris`, `sami`, and `triplespec` in addition to `goodman`, `soi`, and `spartan`, only the latter 3 appear to have recordings from the time period in which I am currently investigating (`2016-02-01 -> 2016-08-01`). Why this is, I cannot say for sure, except that some instruments may have been retired before, or installed after, my selected time period. This should be taken into account if any time periods outwith this one are investigated.

## Download Data

In [19]:
search_parameters = [
    ["telescope", "soar"],
    ["caldat", "2016-02-01", "2016-08-01"],
    ["instrument", "goodman", "soi", "spartan"],
    ["proc_type", "raw"]
]

### Download Core File Information

In [20]:
core_data_filename = "soar_core_data.json"

if False and os.path.isfile(core_data_filename):
    # File already exists, use it instead of redownloading all the data
    core_results = json.load(open(core_data_filename, "r"))

else:    
    # File doesn't exist. Download data from server.
    # Get all core file fields to use as outfields
    r_core_file_fields = requests.get(f"{url_root}/api/adv_search/core_file_fields")

    outfields = [entry['Field'] for entry in r_core_file_fields.json()]

    # Initiate request for all entries from the SOAR telescope between 2016-02-01 and 2016-08-01
    jj = {
        "outfields" : outfields,
        "search" : search_parameters
    }
    core_results = requests.post(f"{url_root}/api/adv_search/fasearch/?limit=1000000", json=jj).json()

    # Save the results to file so they don't have to be redownloaded next time
    core_output = json.dumps(core_results)
    with open(core_data_filename, "w") as f:
        f.write(core_output)

#### Check Core Data

In [ ]:
# Check the value types / ranges for each variable

var_dict = {}

for frame in core_results:
    for var in frame:
        if not (var in var_dict):
            var_dict[var] = {}
        value = frame[var]
        if value in var_dict[var]:
            var_dict[var][value] += 1
        else:
            var_dict[var][value] = 1

var_counts = sorted([(var, len(var_dict[var])) for var in var_dict], key=lambda x: x[1])

print("CONSTANTS")
for v in var_counts:
    if v[1] == 1:
        print(f"{v[0]}: '{list(var_dict[v[0]].keys())[0]}'")
print()

print("CATEGORIES")
for v in var_counts:
    if v[1] > 1 and v[1] <= 20:
        print(f"{v[0]}({v[1]}): {var_dict[v[0]]}")
print()

print("CONTINUOUS")
for v in var_counts:
    if v[1] > 20:
        print(f"{v[0]}: {v[1]} entries")
print()

In [ ]:
# Examine a single entry from the request

core_results[:2]

### Download Auxiliary File Data

In [21]:
aux_data_filename = "soar_aux_data.json"

if False and os.path.isfile(aux_data_filename):
    aux_results = json.load(open(aux_data_filename, "r"))

else:
    # File doesn't exist. Download data from server.
    # Isolate mutual keys for all instruments found in selected time period
    goodman_fields = set(field['Field'] for field in requests.get(f"{url_root}/api/adv_search/aux_file_fields/goodman/raw").json())
    soi_fields = set(field['Field'] for field in requests.get(f"{url_root}/api/adv_search/aux_file_fields/soi/raw").json())
    spartan_fields = set(field['Field'] for field in requests.get(f"{url_root}/api/adv_search/aux_file_fields/spartan/raw").json())

    mutual_fields = sorted(field for field in spartan_fields if ((field in goodman_fields) and (field in soi_fields)))
    mutual_fields

    # Initiate data download
    jj = {
        "outfields" : mutual_fields,
        "search" : search_parameters
    }

    aux_results = requests.post(f"{url_root}/api/adv_search/fasearch/?limit=1000000", json=jj).json()

    # Save to file to prevent having to redownload
    aux_output = json.dumps(aux_results)
    with open(aux_data_filename, "w") as f:
        f.write(aux_output)


In [22]:
len(aux_results), len(core_results)

(25744, 25744)

#### Check compatibility with Core Data

In [23]:
# Ensure that the same file URLs are in each set of results
core_url_set = set([entry['url'] for entry in core_results])
aux_url_set = set([entry['url'] for entry in aux_results])
print("Results match: {}".format(aux_url_set == core_url_set))

Results match: True


## Convert to Dataframe

### Load and merge data

Load each `JSON` file into a `pandas` dataframe, with the index being each row's URL value (as these should be unique, and identical between the two files).

In [24]:
df_core = pd.read_json("soar_core_data.json").set_index("url")
df_aux = pd.read_json("soar_aux_data.json").set_index("url")
df = df_core.join(df_aux)
df.head()

,archive_filename,caldat,date_obs_max,date_obs_min,dec_max,dec_min,depth,exposure,filesize,ifilter,...,OBJECT,OBSERVAT,OBSERVER,OBSID,OBSTYPE,ODATEOBS,PROCTYPE,PRODTYPE,ROTATOR,TELESCOP
url,,,,,,,,,,,,,,,,,,,,,
https://astroarchive.noao.edu/api/retrieve/00052a8dfb863561838b1e4919e23656/,/net/archive/mtn/20160317/soar/soar/psg_160318...,2016-03-17,2016-03-19,2016-03-18,-2.010906,-2.010906,NaN,NaN,535680,<NO FILTER>,...,,SOAR,,None,FLAT,2016-03-18T00:18:18.80,raw,image,353.386,SOAR 4.1m
https://astroarchive.noao.edu/api/retrieve/000624d14b45747a55a0f12c8e3c59e5/,/net/archive/mtn/20160305/soar/soar/psg_160305...,2016-03-05,2016-03-06,2016-03-05,-17.730861,-17.730861,NaN,NaN,1382400,<NO FILTER>,...,focus sequence,SOAR,,None,OBJECT,2016-03-05T18:58:35.94,raw,image,0.000,SOAR 4.1m
https://astroarchive.noao.edu/api/retrieve/000af0e2626b794c1c33e82201d75db2/,/net/archive/mtn/20160614/soar/2016A-0607/psi_...,2016-06-14,2016-06-16,2016-06-15,-35.643054,-35.643054,NaN,120.0,717120,None,...,ECL114_B,SOAR,Corral Santos,soar.sam.20160615T045303.2Z,OBJECT,2016-06-15,raw,image,153.240,SOAR telescope
https://astroarchive.noao.edu/api/retrieve/0012178bf7747a399cdc66d4abdb4b85/,/net/archive/mtn/20160214/soar/soar/psg_160215...,2016-02-14,2016-02-16,2016-02-15,-29.660195,-29.660195,NaN,NaN,144000,g-SDSS,...,wd 1124-293,SOAR,,None,OBJECT,2016-02-15T06:16:10.72,raw,image,346.116,SOAR 4.1m
https://astroarchive.noao.edu/api/retrieve/00123e0058a674a1e1b7d23e7483129a/,/net/archive/mtn/20160205/soar/soar/psg_160206...,2016-02-05,2016-02-07,2016-02-06,15.137767,15.137767,NaN,NaN,512640,<NO FILTER>,...,5345-56010-0846_acq,SOAR,,None,OBJECT,2016-02-06T04:07:37.81,raw,image,269.971,SOAR 4.1m


In [25]:
print("df_core:", df_core.shape, "\ndf_aux:", df_aux.shape, "\nfinal df:", df.shape)

df_core: (25744, 26) 
df_aux: (25744, 24) 
final df: (25744, 50)


### Select relevant columns to keep

The following section contains tests remove as many unnecessary columns as possible.

Final Columns:
* `dec_min` (declintation of target / telescope)
* `ifilter` (filter used)
* `instrument` (instrument used)
* `obs_type` (type of exposure)
* `ra_min` (right ascension of target / telescope)
* `proposal` (proposal ID for exposure)
* `OBJECT` (name / ID of target object)
* `DATE-OBS` (date and time of start of exposure)
* `EXPTIME` (exposure length) - `exposure` had too many Null values

Note: `OBSERVER` (observer performing exposure) was not selected as 63% of entries were empty strings.

#### Summary

Duplicate Columns (some are duplicates in meaning, if not EXACT formatting):
* `ifilter` vs `FILTER`
* `instrument` vs `DTINSTRU`, `INSTRUME`
* `obs_type` vs `OBSTYPE`
* `telescope` vs `DTTELESC`, `TELESCOP`
* `exposure` vs `EXPTIME`
* `DATE-OBS` vs `date_obs_max`, `date_obs_min`, `caldat`, `DTCALDAT`, `ODATEOBS`
(DATE-OBS was chosen because it has the most temporal precision).
* `dec_min` vs `dec_max` (always identical)
* `ra_min` vs `ra_max` (always identical)

Additionally, it might look like there are some disparities between variables that should be constant, like `OBSERVAT` (the Observatory that the exposure was taken at), but that's just because different instruments record these variables differently for some reason ('SOAR' vs 'Cerro Pachon').

#### Tests

In [ ]:
# Check for duplicate columns

def check_duplicate_columns(col1, col2):
    non_duplicates = 0
    for c1, group in df.groupby(col1):
        c2_list = group[col2].unique()
        if len(c2_list) > 1:
            print(c, c2_list)
        else:
            if c1 != c2_list[0]:
                print(c1, c2_list[0])
                
def test_duplicate_columns(col1, col2):
    for c1, group in df.groupby(col1):
        c2_list = group[col2].unique()
        if len(c2_list) > 1 or c1 != c2_list[0]:
            print("({} vs {}): FAILED!".format(col1, col2))
            return
    print("({} vs {}): Duplicates".format(col1, col2))
          
tests = (
    ("caldat", "exposure"),
    ("caldat", "DTCALDAT"),
    ("exposure", "EXPTIME"),
    ("instrument", "DTINSTRU"),
    ("instrument", "INSTRUME"),
    ("proposal", "DTPROPID"),
    ("ifilter", "FILTER"),
    ("obs_type", "OBSTYPE"),
    ("telescope", "DTTELESC"),
    ("telescope", "TELESCOP"),
    ("site", "DTSITE"),
    ("DATE-OBS", "ODATEOBS"),
    ("dec_min", "dec_max"),
    ("ra_min", "ra_max")
)
        
for col1, col2 in tests:
    test_duplicate_columns(col1, col2)

In [ ]:
# Check for disparities between 'instrument' and 'INSTRUME'
for instr, group in df.groupby("instrument"):
    print(instr, group['INSTRUME'].unique())
# They are identical, they just have different formats for the name (abbrev. vs full name)

In [ ]:
# Check for disparities between 'obs_type' and 'OBSTYPE'
for obstype, group in df.groupby("obs_type"):
    print(obstype, group['OBSTYPE'].unique())
# Looks like they are identical, just differently formatted for different instruments

In [ ]:
# Check for disparities between 'telescope' and 'TELESCOP'
for telescope, group in df.groupby("telescope"):
    print(telescope, group['TELESCOP'].unique())
# Again, identical, just different formats for different instruments

In [ ]:
# Check what these columns represent

for c in ("survey", "obs_mode", "proposal", "OBSERVAT", "OBSERVER", "OBSID"):
    uniques = df[c].unique()
    print(c, len(uniques))
    print(uniques)
    print()

In [ ]:
# Check that OBSERVAT is identical WITHIN instruments
# Different instruments record the same data in different formats, for some silly reason

for key, group in df.groupby("OBSERVAT"):
    print(key)
    print(group['instrument'].unique())

In [ ]:
# Determine which Date column is the best for taking as the start of the exposure

for c in ("caldat", "date_obs_max", "DATE-OBS", "ODATEOBS"):
    print(c)
    print(df[c].unique())
    print()
    
# DATE-OBS seems to be the column that consistently has the highest temporal precision

In [ ]:
# Using a known example of ODATEOBS having low precision,
# check that DATE-OBS has higher precision for that date.

df[ df['ODATEOBS'] == '2016-06-15']['DATE-OBS']

### Export Dataframe

Cleanup the dataframe, rename columns, and save it to file.

In [96]:
final_columns = ["DATE-OBS", "instrument", "obs_type", "ifilter", "dec_min", 
                 "ra_min", "OBJECT", "EXPTIME", "proposal"]

all_data = df[final_columns].copy()

data = all_data.rename(columns={"dec_min": "dec", "ra_min": "ra", "OBJECT": "object",
                       "DATE-OBS": "date_obs", "EXPTIME": "exptime"}).copy()

data = data[ data['obs_type']=='object' ]

data = data.groupby('date_obs').nth(0).copy()

data = data.drop(columns="obs_type").reset_index().copy()

data = data.sort_values("date_obs").copy()

data['date_obs'] = data['date_obs'].astype("datetime64")
data["date"] = data['date_obs'].dt.strftime("%Y-%m-%d")
data['time'] = data['date_obs'].dt.time

In [97]:
data

,date_obs,instrument,ifilter,dec,ra,object,exptime,proposal,date,time
0,2016-02-01 19:35:41.025,spartan,H,-17.638404,51.230192,dflat_HOn,26.99362,soar,2016-02-01,19:35:41.025000
1,2016-02-01 19:37:39.369,spartan,H,-17.637906,51.723587,dflat_HOn,26.99362,soar,2016-02-01,19:37:39.369000
2,2016-02-01 19:38:16.790,spartan,H,-17.637751,51.878167,dflat_HOn,26.99362,soar,2016-02-01,19:38:16.790000
3,2016-02-01 19:38:54.197,spartan,H,-17.637586,52.035679,dflat_HOn,26.99362,soar,2016-02-01,19:38:54.197000
4,2016-02-01 19:39:31.619,spartan,H,-17.637425,52.192762,dflat_HOn,26.99362,soar,2016-02-01,19:39:31.619000
5,2016-02-01 19:40:09.040,spartan,H,-17.637267,52.347758,dflat_HOn,26.99362,soar,2016-02-01,19:40:09.040000
6,2016-02-01 19:40:46.447,spartan,H,-17.637103,52.503600,dflat_HOn,26.99362,soar,2016-02-01,19:40:46.447000
7,2016-02-01 19:41:23.869,spartan,H,-17.636942,52.659850,dflat_HOn,26.99362,soar,2016-02-01,19:41:23.869000
8,2016-02-01 19:42:01.290,spartan,H,-17.636779,52.816937,dflat_HOn,26.99362,soar,2016-02-01,19:42:01.290000
9,2016-02-01 19:42:38.697,spartan,H,-17.636617,52.973187,dflat_HOn,26.99362,soar,2016-02-01,19:42:38.697000


#### Corrections

In [76]:
no_dupe_data = data.groupby("date_obs").nth(0)

In [77]:
len(no_dupe_data)

15034

In [36]:
d2['ifilter'] = d2['ifilter'].replace({"<NO FILTER>": None})

In [37]:
for c in data.columns:
    print(c, data[c].isnull().sum())

date_obs 0
instrument 0
ifilter 1746
dec 0
ra 0
object 0
exptime 0
proposal 0
date 0
time 0


In [52]:
print(len(data.columns))
c = data.columns[5]
print(c)
data[c].value_counts()

10
object


GD133                  1220
2264-53682-0527        1161
WD 1150-153            1084
wd 1124-293            1022
2054-53431-0272         799
hwvir_g                 698
GD 133                  608
wd J1221p1245           570
HS J1249+0426           566
5365-55945-0464         435
0474-52000-0365         402
D Flat                  340
3826-555563-0836        296
S708D                   272
focus                   216
3815-55537-0123         201
3233-54891-0636         201
0327-52294-0526         200
0585-52027-0597         181
2530-53881-0307         177
D Flat off              176
Dome flat [FeII]        176
Dome flat K             160
5345-56010-0846_acq     140
flat                    124
                        121
D Flat Off              120
flat_off                120
HE0412                  108
5891-56034-0548         100
                       ... 
MWSC5319-4                1
MWSC5319-3                1
R457323                   1
MWSC5319-5                1
MWSC5323-5          

In [54]:
data[ data['object'].str.contains("flat") ]['object'].unique()

array(['dflat_HOn', 'dflat_JOn', 'dflat_KOn', 'dflat_KOff', 'dflat_HOff',
       'dflat_JOff', 'dflat K', 'dflat H', 'dflat J', 'dflat_K',
       'dflat_H', 'dflat_J', 'dflat_H2', 'dflat_C2', 'dflat_BG',
       'Dome flat K', 'Dome flat H', 'Dome flat J', 'Dome flat [FeII]',
       'Dome flat H2', 'flat', 'flat_off'], dtype=object)

In [75]:
# Duplicate values only occur for the Spartan imager, and always appear to be in 4s
# The Spartan imager has 4 focal planes (?), so it seems reasonably to expect it to have 4 data files.

counter = 0
counter_max = 100000
unique_values = {}
for date, group in df.groupby("DATE-OBS"):
    if len(group) > 1:
        instruments = group['instrument'].unique()
        if len(instruments) == 1 and instruments[0] == 'spartan':
            continue
        for c in group.columns:
            if len(group[c].unique()) > 1:
                if c in unique_values:
                    unique_values[c] += 1
                else:
                    unique_values[c] = 1
        counter += 1
        print(group['instrument'])
        if counter >= counter_max:
            break
            
print(unique_values)

{}


In [67]:
for c in group.columns:
    print(c, len(group[c].unique()))

archive_filename 4
caldat 1
date_obs_max 1
date_obs_min 1
dec_max 1
dec_min 1
depth 1
exposure 1
filesize 4
ifilter 1
instrument 1
md5sum 4
obs_mode 1
obs_type 1
original_filename 4
proc_type 1
prod_type 1
proposal 1
ra_max 1
ra_min 1
release_date 1
seeing 1
site 1
survey 1
telescope 1
updated 4
BZERO 1
CONSWV 1
DATE-OBS 1
DETSIZE 1
DTACQNAM 4
DTCALDAT 1
DTINSTRU 1
DTPROPID 1
DTSITE 1
DTTELESC 1
ENVTEM 1
EXPTIME 1
FILTER 1
INSTRUME 1
OBJECT 1
OBSERVAT 1
OBSERVER 1
OBSID 1
OBSTYPE 1
ODATEOBS 1
PROCTYPE 1
PRODTYPE 1
ROTATOR 1
TELESCOP 1


In [71]:
group.reset_index()['DTACQNAM']

0    dflat_K012-2011d1.fits.fz
1    dflat_K012-2011d0.fits.fz
2    dflat_K012-2011d2.fits.fz
3    dflat_K012-2011d3.fits.fz
Name: DTACQNAM, dtype: object

In [30]:
d2 = data.copy()
d2['ifilter'] = d2['ifilter'].replace({"<NO FILTER>": None})
d2['ifilter'].value_counts()

J           1240
K           1228
g-SDSS      1046
H            900
H2           388
Cont2        268
[FeII]       176
HeI/CIV      136
r-SDSS        68
BrGamma       60
CO            48
Cont3         44
H-alpha       20
u-SDSS        18
z-SDSS         6
i-SDSS         6
B-Bessel       1
Name: ifilter, dtype: int64

In [34]:
d2[ d2['ra'].isnull() ]

# May have to look into removing duplicate exposures???

,date_obs,instrument,ifilter,dec,ra,object,exptime,proposal,date,time
1399,2016-01-27 19:23:38.681,spartan,H2,NaN,NaN,dummy,11.964910,soar,2016-01-27,19:23:38.681000
1400,2016-01-27 19:23:38.681,spartan,H2,NaN,NaN,dummy,11.964910,soar,2016-01-27,19:23:38.681000
1401,2016-01-27 19:23:38.681,spartan,H2,NaN,NaN,dummy,11.964910,soar,2016-01-27,19:23:38.681000
1402,2016-01-27 19:23:38.681,spartan,H2,NaN,NaN,dummy,11.964910,soar,2016-01-27,19:23:38.681000
1403,2016-01-27 19:32:15.103,spartan,H2,NaN,NaN,,0.456786,soar,2016-01-27,19:32:15.103000
1404,2016-01-27 19:32:15.103,spartan,H2,NaN,NaN,,0.456786,soar,2016-01-27,19:32:15.103000
1405,2016-01-27 19:32:15.103,spartan,H2,NaN,NaN,,0.456786,soar,2016-01-27,19:32:15.103000
1406,2016-01-27 19:32:15.103,spartan,H2,NaN,NaN,,0.456786,soar,2016-01-27,19:32:15.103000
2591,2016-01-29 02:09:17.540,spartan,Cont2,NaN,NaN,NGC1316,0.456786,soar,2016-01-29,02:09:17.540000
2592,2016-01-29 02:09:17.540,spartan,Cont2,NaN,NaN,NGC1316,0.456786,soar,2016-01-29,02:09:17.540000


In [31]:
d2['ifilter'].isnull().sum()

14888

## TODO

Replace all null values that are text (e.g. `<NO FILTER>` in `ifilter`) with ACTUAL null values

In [98]:
data.to_pickle("soar_data.pkl")